In [1]:
import pandas as pd
import re
import os
import email
import dateutil.parser as dateparser
import datetime

In [2]:
# Constants
# Tested and verified - small: lokey-t, south-s
user = "zipper-a" # NOTE: SOME PEOPLE DO NOT HAVE THE SAME REPLY FORMAT. 
send_folders = ["_sent_mail", "sent", "sent_items"] # do not edit this!

In [3]:
# Get the number of emails in user's folder. 
def get_num_sent_emails(user, foldername):
    n = (len(os.listdir("data/maildir/"+user+"/"+foldername)))
    return n

In [4]:
def get_user_email_id(user):
    send_folder = send_folders[0] # doesn't actually matter which folder we use.
    # get the "from" field of one of the sent emails
    sentfilename = os.listdir("data/maildir/" + user + "/" + send_folder)[0]
    fullfilename = "data/maildir/" + user + "/" + send_folder + "/" + sentfilename
    with open(fullfilename, "r") as f:
        for line in f:
            if line.startswith("From:"):
                return line.split(": ")[1].strip()
            

# print("Email ID for user {}: ".format(user), get_user_email_id(user))

In [5]:
## Make the dataframe, to be filled with data. 
df = pd.DataFrame(columns=["uid", "sender", "recipient", "sendtime", "replytime", \
                           "replyspeed", "messagecontent", "foldername"])
df.set_index("uid", inplace=True)
# Indices of the dataframe are messageIDs. (because properness.)

In [6]:
# Choose which sent folder to use.
sent_folder_name = ""
for folder in send_folders:
    if os.path.isdir("data/maildir/"+user+"/"+folder):
        if sent_folder_name == "":
            sent_folder_name = folder
        if get_num_sent_emails(user, folder) > get_num_sent_emails(user, sent_folder_name):
            sent_folder_name = folder


print("Using folder: ", sent_folder_name)


Using folder:  sent_items


In [7]:
def isReply(content):    
    # REGARDLESS of whether we have a fwd or a reply, we are looking for a block 
    # that looks like this:
    
    # SENDER on DATE 05/16/2001 05:30:21 PM
    # To: SOMEONE
    # cc: ??
    # Subject: xxx
    time_regex = "\d+:\d\d(|:\d\d)( |)(AM|PM)"
    date_slash_regex = "\d\d/\d\d/20\d\d " + time_regex
    whitespace_regex = r"[\s(=20)(=09)]+"
    a = re.search(date_slash_regex + whitespace_regex + r"To:", content)
    if a is not None:
        just_date = re.search(date_slash_regex, a.group()).group()
        return just_date
    
    # Format:
    # -----Original Message-----
    # From: whatever
    # Sent: Friday, October 05, 2001 3:07 PM
    day_regex = r"(\w+)day, \w+ \d\d, 20\d\d " + time_regex
    a = re.search(day_regex, content)
    if a is not None:
        return a.group()
        


    else:
        return -1



In [8]:
# Iterate through the sent emails and add to dataframe. 
filenames = os.listdir("data/maildir/"+user+"/"+sent_folder_name)
for file in filenames:
    full_filename = "data/maildir/"+user+"/"+sent_folder_name+"/"+file
    f = open(full_filename, "r")

    # Process the message
    message = email.message_from_file(f)
    f.close()

    # Get the list of recipients.
    # print(message.items())
    if message["To"]:
        recipient_list = message["To"].split(", ")
    if message["Cc"]:
        recipient_list += message["Cc"].split(", ")
    if message["Bcc"]:
        recipient_list += message["Bcc"].split(", ")
    # if message["X-To"]:
    #     recipient_list += message["X-To"].split(", ")
    # if message["X-cc"]:
    #     recipient_list += message["X-cc"].split(", ")
    # if message["X-bcc"]:
        recipient_list += message["X-bcc"].split(", ")
    recipient_list = list(set([r.strip() for r in recipient_list])) # remove duplicates
    if len(recipient_list) == 0:
        continue # Email not sent to anyone?

    # Get the time the email was sent.
    sendtime = dateparser.parse(message["Date"])
    # print("send time", sendtime.astimezone(datetime.timezone.utc))

    # Calculate the reply speed:
    prev_email_time = isReply(content=message.get_payload())
    if prev_email_time != -1:
        # print("reply time", prev_email_time)
        prev_email_time = dateparser.parse(prev_email_time + "+0000")
        # print("prev_email_time", prev_email_time.astimezone(datetime.timezone.utc))
        replyspeed = sendtime - prev_email_time
        # print("replyspeed", replyspeed)
    else:
        replyspeed = -1

    print(message.get_payload()[:300])
    print("---")

    # Make dataframe row(s):
    for recipient in recipient_list:
        row = {
            "uid": message["Message-ID"]+recipient,
            "sender": message["From"],
            "recipient": recipient,
            "sendtime": sendtime,
            "replytime": prev_email_time, 
            "replyspeed": replyspeed,
            "messagecontent": message.get_payload(),
            "foldername": ""
        }
        id = message["Message-ID"]+recipient,
        entry = pd.DataFrame([row])
        df = pd.concat([df, entry], ignore_index=True)

df

    


Thanks, I think I am okay, but let's talk tonight.
---
The Lions win !    The Lions win !

Break up the Lions....

Andy
---


 

--------- Inline attachment follows ---------

From:  </O=ENRON/OU=NA/CN=RECIPIENTS/CN=LCANNO2>
Date: Tuesday, May 15, 2001 1:40:16 GMT
Subject: 
---
experian 1-888-397-3742 done conf#3267747298

equifax  1-800-525-6285

transunion  1-800-916-8800
                 1-800-680-7289
---
Thanks for that, any luck on anything in '03 ?

---
Can I get a Reuters screen as well ?
---
I know Steve Murray quite well.

---
I asked Steve and here is what he sent me.

-----Original Message-----
From: Meyer, Steve (Exchange) [mailto:smeyer@bear.com]
Sent: Sunday, January 06, 2002 8:42 PM
To: Zipper, Andy
Subject: RE: Hi


Hi Andy and happy new year...

I chose one in NJ for my own use.

I did get some recommendations fro
---
I keep getting asked for a username,password and domain name. What do I put in these fields to get access ?

---
Sonjay,

Please give me a call at your 

/tmp/ipykernel_207134/905744102.py:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, entry], ignore_index=True)


Justin,
The traders are free to use any platform they like. I do not approve or disapprove; they are in the best position to make that decision. It is EWS policy however that someone from Amita/Lara's group be the master user to montor activity and control what access agreements we accept.

Thanks,

---
That would be great.

---
same plan

---


-----Original Message-----
From: mitchell24@att.net [mailto:mitchell24@att.net]
Sent: Wednesday, September 26, 2001 8:14 AM
To: Ben Natali
Subject: REQUIRED READING (fwd)



----------------------  Forwarded Message:  ---------------------

 
Who did it?     Foreign Report presents an alternative 
---
You're right he does look good. I am very interested. Can you set up a phone interview, and if that goes well bring him down for rounds.

---
Mike is back at his desk this morning.
---
COORD ADMIN (713) 853-7257 
---
When will we have a draft available fo distribution to Accenture ?

---
my work phone is 713 853 6994

---
would you like me to fill

,sender,recipient,sendtime,replytime,replyspeed,messagecontent,foldername,uid
0,andy.zipper@enron.com,lottyplace@yahoo.com,2001-11-28 09:58:42-08:00,-1,-1,"Thanks, I think I am okay, but let's talk toni...",,<8611097.1075859061118.JavaMail.evans@thyme>lo...
1,andy.zipper@enron.com,john_bourget@bankone.com,2002-01-07 07:13:42-08:00,-1,-1,The Lions win ! The Lions win !\n\nBreak up...,,<16898807.1075842017414.JavaMail.evans@thyme>j...
2,andy.zipper@enron.com,s..palmer@enron.com,2001-05-23 10:05:39-07:00,-1,-1,\n\n \n\n--------- Inline attachment follows -...,,<17891065.1075845422242.JavaMail.evans@thyme>s...
3,andy.zipper@enron.com,aazccb@swbell.net,2001-05-24 10:07:58-07:00,-1,-1,experian 1-888-397-3742 done conf#3267747298\n...,,<25014417.1075845422443.JavaMail.evans@thyme>a...
4,andy.zipper@enron.com,steve.wang@enron.com,2001-10-08 10:01:04-07:00,-1,-1,"Thanks for that, any luck on anything in '03 ?\n",,<23654783.1075851548753.JavaMail.evans@thyme>s...
...,...,...,...,...,...,...,...,...
434,andy.zipper@enron.com,claudio.ribeiro@enron.com,2001-06-07 13:04:01-07:00,-1,-1,"Claudio,\n\nI agree with you. The first bullet...",,<32601344.1075845425088.JavaMail.evans@thyme>c...
435,andy.zipper@enron.com,scotts@bennettjones.ca,2002-01-29 12:06:58-08:00,-1,-1,Why is Mr. Frank Alexander interested intalkin...,,<24442791.1075842016887.JavaMail.evans@thyme>s...
436,andy.zipper@enron.com,robert.bogucki@enron.com,2001-11-07 07:34:43-08:00,-1,-1,do u have msn instant messenger ?\n,,<19505961.1075861170661.JavaMail.evans@thyme>r...
437,andy.zipper@enron.com,lydia.cannon@enron.com,2001-05-29 11:03:39-07:00,2001-05-29 13:02:00+00:00,0 days 05:01:39,\nKal will be finishing mine shortly. Please b...,,<11160116.1075845423048.JavaMail.evans@thyme>l...


In [9]:
# SAVE AS CSV
!mkdir -p processed_data
df.to_csv("processed_data/"+user+"_sent.csv")